# `quickmt` Demo

First, download a model:

```bash
quickmt-model-download quickmt/quickmt-zh-en ./quickmt-zh-en
```

In [ ]:
from pathlib import Path

import datasets
from quickmt import Translator
from sacrebleu import BLEU, CHRF

In [ ]:
# For CPU inference
# t = Translator("../quickmt-models/quickmt-zh-en", device="cpu", intra_threads=6, inter_threads=1)

# Will auto-detect GPU if available by default
t = Translator(
    "/home/mark/Downloads/ct2-enfr/", device="cuda", intra_threads=1, inter_threads=1
)

In [ ]:
# Can input a string
t(
    "Birds are very interesting and so is Juno. Kakapoes are almost extinct and are very rare.",
    beam_size=5,
)

In [ ]:
t("My name is Azura")

In [ ]:
t("Fizzzzzz")

In [ ]:
t("It is almost supper time")

In [ ]:
t("Cat.", beam_size=5)

In [ ]:
t("It is winter")

In [ ]:
t("Today is Friday, I love weekends!")

In [ ]:
# Or a list of strings
# Change max_batch_size so that batches will fit in RAM on your machine
t(
    ["他补充道：“我们现在有 4 个月大没有糖尿病的老鼠，但它们曾经得过该病。”"] * 8,
    beam_size=5,
    max_batch_size=3,
)

In [ ]:
# Get alternative translations by sampling
# Can pass any CTranslate2 Translator.translate_batch arguments
# See https://opennmt.net/CTranslate2/python/ctranslate2.Translator.html#ctranslate2.Translator.translate_batch
t(
    ["他补充道：“我们现在有 4 个月大没有糖尿病的老鼠，但它们曾经得过该病。”"],
    sampling_temperature=0.8,
    beam_size=1,
    sampling_topk=50,
    sampling_topp=0.99,
)

## Evaluate on flores-devtest

In [ ]:
bleu = BLEU()
chrf = CHRF()
src_lang = "zho_Hans"
tgt_lang = "eng_Latn"

try:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{src_lang}-{tgt_lang}",  # trust_remote_code=True
    )
except:
    flores = datasets.load_dataset(
        "facebook/flores",
        f"{tgt_lang}-{src_lang}",  # trust_remote_code=True
    )

src = []
ref = []
for i in flores["devtest"]:
    src.append(i[f"sentence_{src_lang}"])
    ref.append(i[f"sentence_{tgt_lang}"])

In [ ]:
results = t(src, beam_size=1)

In [ ]:
results[:5]

In [ ]:
print(bleu.corpus_score(results, [ref]))
print(chrf.corpus_score(results, [ref]))